In [11]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

all_colors = px.colors.qualitative.Plotly

In [69]:
runs = [
    # (  RUN NAME,                DF,                                              INFO  )
    # ("CPU Baseline", [pd.read_csv("res/1.1.benchmark_results_cpu.csv", header=0)], (1,)),
    ("1 Thread", [pd.read_csv("res/6.benchmark_results_cpu_th_1.csv", header=0)], (1,)),
    # ("2 Threads", [pd.read_csv("res/6.benchmark_results_cpu_th_2.csv", header=0)], (2,)),
    # ("4 Threads", [pd.read_csv("res/6.benchmark_results_cpu_th_4.csv", header=0)], (4,)),
    # ("8 Threads", [pd.read_csv("res/6.benchmark_results_cpu_th_8.csv", header=0)], (8,)),
    # ("12 Threads", [pd.read_csv("res/6.benchmark_results_cpu_th_12.csv", header=0)], (12,)),
    # ("16 Threads", [pd.read_csv("res/6.benchmark_results_cpu_th_16.csv", header=0)], (16,)),
    # ("CPU Reduction", [pd.read_csv("res/7.1.benchmark_results_cpu_th_16.csv", header=0)], (16,)),
    # ("CPU Critical", [pd.read_csv("res/7.2.benchmark_results_cpu_th_16.csv", header=0)], (16,)),
    # ("16 Threads, 1D", [pd.read_csv("res/16.benchmark_results_cpu_th_16.csv", header=0)], (16,)),

    # # GPU optimizations
    # ("GPU", [pd.read_csv("res/1.2.benchmark_results_gpu.csv", header=0)], (8704,)),
    # ("GPU L1 opt", [pd.read_csv("res/2.benchmark_results_gpu_cache_l1.csv", header=0)], (8704,)),
    # ("GPU No Branch", [pd.read_csv("res/3.benchmark_results_gpu_no_branching.csv", header=0)], (8704,)),
    # ("GPU K Config", [pd.read_csv("res/4.benchmark_results_gpu_config.csv", header=0)], (8704,)),
    # ("GPU", [pd.read_csv("res/5.benchmark_results_gpu_pre_inc.csv", header=0)], (8704,)),
    # ("GPU Multi NH", [pd.read_csv("res/8.benchmark_results_gpu.csv", header=0)], (8704,)),
    # ("GPU Multi NH Unroll", [pd.read_csv("res/9.benchmark_results_gpu.csv", header=0)], (8704,)),
    # ("GPU NH Radius 2", [pd.read_csv("res/10.benchmark_results_gpu.csv", header=0)], (8704,)),
    # ("GPU NH Radius 3", [pd.read_csv("res/11.benchmark_results_gpu.csv", header=0)], (8704,)),
    # ("GPU No Opt", [pd.read_csv("res/12.benchmark_results_gpu.csv", header=0)], (8704,)),
    # ("GPU Opt 3", [pd.read_csv("res/13.benchmark_results_gpu.csv", header=0)], (8704,)),
    # ("GPU Opt 2", [pd.read_csv("res/14.benchmark_results_gpu.csv", header=0)], (8704,)),
    # ("GPU GridType", [pd.read_csv("res/15.benchmark_results_gpu.csv", header=0)], (8704,)),
    ("GPU 2D, K fixed", [
        pd.read_csv("res/17.1.benchmark_results_gpu.csv", header=0),
        pd.read_csv("res/17.2.benchmark_results_gpu.csv", header=0),
        pd.read_csv("res/17.3.benchmark_results_gpu.csv", header=0),
        pd.read_csv("res/17.4.benchmark_results_gpu.csv", header=0),
        pd.read_csv("res/17.5.benchmark_results_gpu.csv", header=0),
    ], (8704,)),
    ("GPU 1D, K opt", [
        pd.read_csv("res/18.1.benchmark_results_gpu.csv", header=0),
        pd.read_csv("res/18.2.benchmark_results_gpu.csv", header=0),
        pd.read_csv("res/18.3.benchmark_results_gpu.csv", header=0),
        pd.read_csv("res/18.4.benchmark_results_gpu.csv", header=0),
        pd.read_csv("res/18.5.benchmark_results_gpu.csv", header=0),
    ], (8704,)),
]

cpu_df = pd.concat(runs[0][1])
cpu_df = cpu_df.groupby(cpu_df.index).mean() # use cpu_df mean only

runs_gpu = runs[1:]

In [70]:
def add_mean_std_trace(fig, df, name, idx, y_col, x_col="size"):
    df_mean = df.groupby(df.index).mean()
    df_std = df.groupby(df.index).std()

    color = all_colors[idx % len(all_colors)]
    fig.add_traces(go.Scatter(
        x=df_mean[x_col],
        y=df_mean[y_col] + df_std[y_col],
        fill='tonexty',
        line=dict(color=color, width=0.1),
        mode="lines",
        name=name + " upper",
        showlegend=False,
    ))
    fig.add_trace(go.Scatter(
        x=df_mean[x_col],
        y=df_mean[y_col],
        fill='tonexty',
        line=dict(color=color),
        mode="lines+markers",
        name=name,
    ))
    fig.add_traces(go.Scatter(
        x=df_mean[x_col],
        y=df_mean[y_col] - df_std[y_col],
        fill='tonexty',
        line=dict(color=color, width=0.1),
        mode="lines",
        name=name + " lower",
        showlegend=False,
    ))

In [71]:
fig = go.Figure()
for idx, (name, dfs, _) in enumerate(runs):
    df = pd.concat(dfs)
    df["avgEvolveTime"] = df["avgEvolveTime"] / 1e6 # time is originally in ns

    add_mean_std_trace(fig, df, name, idx, "avgEvolveTime")

fig.update_layout(title="Average grid evolution time", xaxis_title='Lattice Size', yaxis_title="Time (ms)", width=500, height=400)
fig.update_yaxes(type="log")
fig.show()

In [72]:
fig = go.Figure()

for idx, (name, dfs, _) in enumerate(runs_gpu):
    for i, df in enumerate(dfs):
        dfs[i]["speedup"] = cpu_df["avgEvolveTime"] / df["avgEvolveTime"]

    df = pd.concat(dfs)
    add_mean_std_trace(fig, df, name, idx + 1, "speedup")

fig.update_layout(title="Speedup", xaxis_title='Lattice Size', yaxis_title="Speedup", width=500, height=400)
# fig.update_yaxes(type="log")
fig.show()

In [73]:
fig = go.Figure()

for i, (name, dfs, info) in enumerate(runs_gpu):
    for i, df in enumerate(dfs):
        dfs[i]["efficiency"] = 100 * (cpu_df["avgEvolveTime"] / df["avgEvolveTime"]) / info[0]
    
    df = pd.concat(dfs)
    add_mean_std_trace(fig, df, name, i + 1, "speedup")

fig.update_layout(title="Efficiency", xaxis_title='Lattice Size', yaxis_title="Efficiency %", width=1000, height=600)
fig.update_yaxes()
fig.show()